In [ ]:
import pandas as pd

df = pd.read_csv('data/data1.tsv', delimiter = '\t')

df

In [ ]:
import numpy as np
from keras.preprocessing.image import load_img, img_to_array

imgs = np.array([ img_to_array(load_img(f'img/{i}')) for i in df['image'].values ])

imgs.shape

In [ ]:
cols = ['x1', 'y1', 'x2', 'y2', 'x3', 'y3', 'x4', 'y4']

dfl = df[cols].astype(np.float64)

for c in cols:
    n = 2 if c.startswith('x') else 1
    dfl[c] = dfl[c] / imgs.shape[n]

labels = dfl.values
labels

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Flatten, Dropout
from keras.layers.convolutional import Conv2D
from keras.layers.pooling import MaxPool2D
from keras.layers.normalization import BatchNormalization

model = Sequential()

model.add(BatchNormalization(input_shape = imgs.shape[1:]))

model.add(Conv2D(6, 3, activation = 'relu'))
model.add(MaxPool2D())

model.add(BatchNormalization())
model.add(Dropout(0.2))

model.add(Conv2D(16, 3, activation = 'relu'))
model.add(Conv2D(16, 3, activation = 'relu'))
model.add(MaxPool2D())

model.add(BatchNormalization())
model.add(Dropout(0.2))

model.add(Conv2D(32, 3, activation = 'relu'))
model.add(Conv2D(32, 3, activation = 'relu'))
model.add(MaxPool2D())

model.add(BatchNormalization())
model.add(Dropout(0.2))

model.add(Conv2D(64, 3, activation = 'relu'))
model.add(Conv2D(64, 3, activation = 'relu'))
model.add(Conv2D(64, 3, activation = 'relu'))
model.add(MaxPool2D())

model.add(BatchNormalization())
model.add(Dropout(0.2))

model.add(Flatten())
model.add(Dense(32, activation = 'relu'))

model.add(BatchNormalization())
model.add(Dropout(0.2))

model.add(Dense(32, activation = 'relu'))

model.add(BatchNormalization())
model.add(Dropout(0.2))

model.add(Dense(8, activation = 'sigmoid'))

model.compile(loss = 'mean_squared_error', optimizer = 'adam', metrics = ['mae'])

model.summary()

In [ ]:
hist = model.fit(imgs, labels, epochs = 50, batch_size = 10)
hist

In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt

plt.plot(hist.history['loss'])

In [ ]:
import cv2

def scale(d):
    for i in range(len(d)):
        n = 2 if i % 2 == 0 else 1
        d[i] *= imgs.shape[n]

    return d
        

def predict(i, s = 8.0):
    plt.rcParams['figure.figsize'] = (s, s)
    
    r = scale(np.copy(labels[i])).astype(np.int32)
    p = scale(model.predict(np.array([imgs[i]]))[0]).astype(np.int32)
   
    img = cv2.imread(f"img/{df['image'][i]}")
    
    print(r)
    print(p)
    
    line_size = 3
    r_color = (0, 255, 0)
    p_color = (0, 0, 255)
    
    img = cv2.line(img, (r[0], r[1]), (r[2], r[3]), r_color, line_size)
    img = cv2.line(img, (r[4], r[5]), (r[6], r[7]), r_color, line_size)
    
    img = cv2.line(img, (p[0], p[1]), (p[2], p[3]), p_color, line_size)
    img = cv2.line(img, (p[4], p[5]), (p[6], p[7]), p_color, line_size)
    
    plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))


In [ ]:
predict(0)

In [ ]:
predict(2)

In [ ]:
predict(3)

In [ ]:
predict(4)

In [ ]:
predict(39)